In [1]:
import sys
import os
ROOT_DIR = os.path.abspath("/MoStress/")
sys.path.append(ROOT_DIR)

In [2]:
import json
from os.path import join

import numpy as np
import pandas as pd
from kerasbeats import prep_time_series
from utils.preprocessingCheckpoint import getPreprocessingCheckpoint
from models.architectures.NBEATS.NBEATSModel import NBEATSModel
from tensorflow import keras


In [3]:
trainData, validationData = getPreprocessingCheckpoint()

In [4]:
dataset = {
    "features": trainData["features"],
    "targets": trainData["targets"],
    "weights": trainData["weights"],
}

In [5]:
inputTest = dataset["features"][0]

inputTest

array([[-0.04906158,  0.32236075,  2.31629804, -0.70362568,  0.35170601],
       [ 0.00304426, -0.309945  ,  2.28104558, -0.45453641,  0.43153586],
       [ 0.62631692,  0.45954869,  2.19640463, -0.31583771,  0.497078  ],
       ...,
       [-0.00608964, -0.49451708,  1.34191815, -2.03553343, -0.74628732],
       [ 0.23747035,  0.34576267,  1.38512626, -1.23622251, -0.75530661],
       [-2.68725794,  1.26433206,  1.45325614, -2.09487152, -0.75049123]])

In [6]:
inputTest.shape

(420, 5)

In [7]:
signalTest = inputTest[ : , 0 ]
signalTest.shape

(420,)

In [8]:
X, y = prep_time_series(signalTest, lookback = 7, horizon = 1)

In [9]:
X.shape, y.shape

((413, 7), (413, 1))

In [10]:
nBeats = NBEATSModel()

In [11]:
nBeats.fit(X, y, epochs = 10, verbose = True, use_multiprocessing=True)

2022-09-29 19:50:16.692361: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: 0.6702 - mae: 0.6702 - mape: 1039.6436
Epoch 2/10
1/1 [==============================] - 0s 299ms/step - loss: 10.7098 - mae: 10.7098 - mape: 14957.9092
Epoch 3/10
1/1 [==============================] - 0s 277ms/step - loss: 2.5895 - mae: 2.5895 - mape: 3029.9153
Epoch 4/10
1/1 [==============================] - 0s 254ms/step - loss: 21.4527 - mae: 21.4527 - mape: 24954.4102
Epoch 5/10
1/1 [==============================] - 0s 271ms/step - loss: 2.2382 - mae: 2.2382 - mape: 3254.9941
Epoch 6/10
1/1 [==============================] - 0s 271ms/step - loss: 0.9186 - mae: 0.9186 - mape: 1384.1648
Epoch 7/10
1/1 [==============================] - 0s 255ms/step - loss: 0.8628 - mae: 0.8628 - mape: 1319.5848
Epoch 8/10
1/1 [==============================] - 0s 278ms/step - loss: 0.7866 - mae: 0.7866 - mape: 983.6898
Epoch 9/10
1/1 [==============================] - 0s 280ms/step - loss: 0.6891 - mae: 0.6891 - mape: 1002.3229

In [12]:
# nBeats.model_layer._residual_collection

In [13]:
class CustomCallback(keras.callbacks.Callback):
    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))
        print(f"\n\n This is the residuals {self.model.get_layer(name = 'nbeats')._residual_collection}")

In [14]:
nBeats.predict(X, callbacks=[CustomCallback()])

...Predicting: end of batch 0; got log keys: ['outputs']


 This is the residuals {0: <tf.Tensor 'model/nbeats/subtract/sub:0' shape=(None, 7) dtype=float32>, 1: <tf.Tensor 'model/nbeats/subtract_1/sub:0' shape=(None, 7) dtype=float32>, 2: <tf.Tensor 'model/nbeats/subtract_2/sub:0' shape=(None, 7) dtype=float32>, 3: <tf.Tensor 'model/nbeats/subtract_3/sub:0' shape=(None, 7) dtype=float32>, 4: <tf.Tensor 'model/nbeats/subtract_4/sub:0' shape=(None, 7) dtype=float32>, 5: <tf.Tensor 'model/nbeats/subtract_5/sub:0' shape=(None, 7) dtype=float32>, 6: <tf.Tensor 'model/nbeats/subtract_6/sub:0' shape=(None, 7) dtype=float32>, 7: <tf.Tensor 'model/nbeats/subtract_7/sub:0' shape=(None, 7) dtype=float32>, 8: <tf.Tensor 'model/nbeats/subtract_8/sub:0' shape=(None, 7) dtype=float32>, 9: <tf.Tensor 'model/nbeats/subtract_9/sub:0' shape=(None, 7) dtype=float32>, 10: <tf.Tensor 'model/nbeats/subtract_10/sub:0' shape=(None, 7) dtype=float32>, 11: <tf.Tensor 'model/nbeats/subtract_11/sub:0' shape=(None

array([[-2.40624830e-01],
       [-2.35612854e-01],
       [-1.17463581e-02],
       [ 3.60860437e-01],
       [ 5.68383408e+00],
       [-6.57068342e-02],
       [ 2.21411422e-01],
       [-4.58423980e-03],
       [-1.63716033e-01],
       [-1.56815737e-01],
       [ 1.60875708e-01],
       [-1.44321740e-01],
       [-4.10637021e-01],
       [ 1.56795800e-01],
       [-4.71797079e-01],
       [-1.63006097e-01],
       [-1.88767016e-01],
       [ 2.16200873e-02],
       [ 1.28829017e-01],
       [-2.19784212e+00],
       [-5.95370047e-02],
       [ 3.03920209e-01],
       [ 3.75112705e-02],
       [-2.59148151e-01],
       [ 7.50209332e-01],
       [ 1.91025287e-01],
       [ 1.55275941e+00],
       [ 1.82194680e-01],
       [ 9.32653427e-01],
       [-1.79966033e-01],
       [-9.56501588e-02],
       [ 2.92758673e-01],
       [-4.56530660e-01],
       [-7.02116191e-01],
       [-6.24191344e-01],
       [ 1.52533829e-01],
       [-6.63376212e-01],
       [-2.44484976e-01],
       [-1.2

In [ ]:
from tensorflow import keras
import tensorflow.keras.backend as K

In [ ]:
X

In [ ]:
resid = K.reverse(X, axes=0)
resid

In [ ]:
teste = keras.layers.Subtract()([resid, X])

In [ ]:
teste

In [ ]:
timeInputLayers = []
nbeatsLayers = []
for i in range(inputTest.shape[1]):
    time_input = keras.layers.Input(shape = (7, ))
    nbeats = NBeatsModel(model_type = 'generic', lookback = 7, horizon = 1).build_layer()
    timeInputLayers.append(time_input)
    nbeatsLayers.append(nbeats.model_layer(time_input))

In [ ]:
concat = keras.layers.Concatenate()(nbeatsLayers)
output = keras.layers.Dense(3, activation="softmax")(concat)
model = keras.Model(inputs = timeInputLayers, outputs = output)

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy')

In [ ]:
x = [ convert_to_tensor(inputTest[:, i]) for i in range(inputTest.shape[1]) ]
y = [
    np.array(
        [
            np.int64(randint(0, 2))
            for _ in range(inputTest.shape[0])
        ]
    )
    for _ in range(inputTest.shape[1])
]

In [ ]:
x = stack(x)
y = convert_to_tensor(stack(y))

In [ ]:
model.fit(x, y, verbose = True)